In [1]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Reshape, Concatenate, Activation, Dense
import keras
import numpy as np
import string
# from progressbar import progressbar
from keras.models import Model

Using TensorFlow backend.


In [8]:
def extract_features():
    # load model
    model = VGG16()
    model.layers.pop()
    # extract final 49x512 conv layer for context vectors
    final_conv = Reshape([49,512])(model.layers[-4].output)
    model = Model(inputs=model.inputs, outputs=final_conv)
    print(model.summary())
    features = dict()

In [9]:
dim_ctx = -1
n_ctx = -1
lstm_cell_dim = -1
lstm_hidden_dim = -1

# Initializes the cell & hidden states of the LSTM by passing the mean context vector through MLPs
# [contexts] : n_ctx x dim_ctx matrix
def init_lstm_states(contexts):
    mean_context = tf.reduce_mean(contexts,)
    
    # num hidden units in MLP
    mlp_hidden = 256
    
    # cell_state MLP
    inputs_c = Input(shape=(dim_ctx,))
    f_c = Dense(mlp_hidden,activation="relu")(inputs_c)
    f_c = Dense(lstm_cell_dim,activation=None)(f_c)
    
    # hidden_state MLP
    inputs_h = Input(shape=(dim_ctx,))
    f_h = Dense(mlp_hidden,activation="relu")(inputs_h)
    f_h = Dense(lstm_hidden_dim,activation=None)(f_h)
    
    return f_c,f_h

# Computes alpha values (attention weights) by passing context vectors + hidden state through MLP
# Includes hidden state by concatenating to end of alpha values
def generate_alphas(contexts,hidden_state):
    mlp_hidden = 256
    
    # tile and concatenate inputs
    batch_size = contexts.shape[0]
    tiled_hidden_state = tf.tile([[hidden_state]],[batch_size,n_ctx,1])
    concat_input = Concatenate(axis=-1)((contexts,tiled_hidden_state))
    
    # feed into MLP
    inputs = Input(shape=(dim_ctx+lstm_hidden_dim,))
    x = Dense(mlp_hidden,activation="relu")(inputs)
    x = Dense(1,activation=None)(x)
    x = Activation("softmax")(x)
    
    return x

# Generates a soft-attention attention vector from alphas & context vectors
def get_soft_attention_vec(contexts,alphas):
    return contexts*tf.reshape(alphas,[1,-1,1])

In [4]:
alphas = np.random.randint(10,size=[49])
contexts = np.ones([10,49,512])

In [5]:
(contexts*np.reshape(alphas,[1,-1,1])).shape

(10, 49, 512)

In [6]:
dim_ctx = 10
lstm_hidden_dim = 10
mlp_hidden = 1
inputs = Input(shape=(dim_ctx+lstm_hidden_dim,))
x = Dense(mlp_hidden,activation="relu")(inputs)
x = Dense(1,activation=None)(x)
x = Activation("softmax")(x)

In [7]:
x

<tf.Tensor 'activation_1/Softmax:0' shape=(?, 1) dtype=float32>